# CNN with PyTorch and MNIST data

The idea will be to buil a MNIST digital classifier using Pytorch.
Images will be 28x28 pixels in grayscale. First layer will consist of 32 channels, each of 5x5 convolutional filetrs and ReLU activation, anda a 2x2 max pooling with a stride of 2. This gives a 14x14 layer output. Then we apply another convolution with 64 channels and 5x5 in size, and another final 2x2 max poolin with stride 2. We end up with a 7x7 output. We will be also flattening the operations, which after the first convolution results in a 7x7x64=3164 nodes, an intermediate layer of 1000 and finally 10 output nodes.

In [13]:
#importing packages 
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch import nn, optim

In [ ]:
#Next we select where to store the MNIST data
DATA_PATH = 'C:\\Users\Marco\Desktop\Python\MachineLearning\Pytorch\MNISTData'
MODEL_STORE_PATH = 'C:\\Users\Marco\Desktop\Python\MachineLearning\Pytorch\pytorch_models\\'

We must first select where the data will be downloaded.

In [ ]:
# transforms to apply to the data.  transforms.Compose()  sale de torchvision
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

Torchvision stores the MNIST dataset, which can be acessed through the DataLoader functionality. In order to process the data, we must first transform it. We use the transforms.Compose() function, which comes inside torchvision and transforms input data into a PyTorch tensor. These tensors are multidimensional matrices. 

Next, the Compose() function acts as a normalization transformation. It maps data in the range -1 to 1 or 0 to 1, so that neural networks can train better. The normalization has two parameters: the mean (0.1307) and the standard deviation (0.3081).  MNIST Data is single channeled, but higher channeled data requires a mean and a standard deviation for each channel.

With the functions _train_dataset_ and _test_dataset_ we download our data and split for training and testing. The root argument specifies the folder that contains train.pt and test.pt. The _train_ argument is a boolean, and informs which data ser to pick between train.pt and test.pt. The _transform_ argument, we specify the transformation (trans in our case). The _download_ argument serves obviusly for downloading the data.

In [14]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Now we implement the DataLoader object, which is a really powerful feature of PyTorch (it allows data shuffling and batching, and loads the data in parallel).

In [15]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out        


Next, we create the Network. This is the core of the program. The easiest way of creating a neural network in PyTorch is by crating a class which inherits the nn.Module, i.e. the class that contains all the necessary for constructing a typical deep learning network within PyTorch.

The nn.Sequential allows us to create sequentially ordered layers in the network: convolution + ReLU + pooling.
The first element is Conv2d, which creates the convolutional filters. Its first argument is the number of input channels (in our case it is 1, because we have single channel greyscale images). The second argument is the number of output channels, which is 32 in our case. The _kernel_size_ is the size of the filter (5x5), and if we wanted non squared shapes, we would have inputed a tupple with the _x-size_ and _y-size_. The padding argument can be calculated from the output size formula:

$\begin{equation}
W_{\rm out} = \frac{W_{\rm in}-F+2P}{S} + 1
\end{equation}$

with $W_{\rm in}$ is the width of the input, _F_ is the filter size, _P_ is the padding and _S_ is the stride. From here we deduce that the padding must be 2.

Next, we perform the ReLU activaton and the pooling. The pooling has as arguments the pooling size, and the stride. There is a third arument in MaxPool2d which is the padding, but is zero as default and in our case it is exactly zero.
The output from _self.layer1_ will be 32 channels of 14x14 images.

The second layer works pretty much the same, but with different siezes. Now the input are 32 channels and the output 64. The output from _self.layer2_ is 64 channels of 7x7.

The drop-out layer is used for avoid overfitting the model. 
Finally, we create two fully connected layers. The first with 7x7x64 nodes and the second of 1000 nodes. In PyTorch, fully connected layers are created via nn.Linear, with the number of nodes as the first argument, and the number of nodes of the following layer as the second argument.

Finally, the forward functions specifies how the data flows through these layers. The name selection is important, because this function will override the base forward function in nn.Module. The argument _x_ is the data and gets passed by. 
After self.layer2 we reshape the function by flattening the 7x7x64 into 3164x1. 

In [16]:
model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In order to train our model, we create an instance of ConvNet(), and define a loss function and an optimizer.
The loss function will be PyTorch's CrossEntropyLoss() function, which combines both a SoftMax activation and a cross entropy loss function.

The optimizer is of the type Adam. We pass the parameters as the first argument (model.parameters() keeps track of these) and the learning rate as the second argument.

In [ ]:
# Hyperparameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

We define the hyperparameters for our training.

In [17]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 0.1265, Accuracy: 96.00%
Epoch [1/5], Step [200/600], Loss: 0.1119, Accuracy: 97.00%
Epoch [1/5], Step [300/600], Loss: 0.0996, Accuracy: 96.00%
Epoch [1/5], Step [400/600], Loss: 0.0571, Accuracy: 98.00%
Epoch [1/5], Step [500/600], Loss: 0.0974, Accuracy: 94.00%
Epoch [1/5], Step [600/600], Loss: 0.0159, Accuracy: 100.00%
Epoch [2/5], Step [100/600], Loss: 0.0489, Accuracy: 99.00%
Epoch [2/5], Step [200/600], Loss: 0.0355, Accuracy: 99.00%
Epoch [2/5], Step [300/600], Loss: 0.1142, Accuracy: 98.00%
Epoch [2/5], Step [400/600], Loss: 0.0961, Accuracy: 97.00%
Epoch [2/5], Step [500/600], Loss: 0.0368, Accuracy: 99.00%
Epoch [2/5], Step [600/600], Loss: 0.0470, Accuracy: 98.00%
Epoch [3/5], Step [100/600], Loss: 0.0208, Accuracy: 99.00%
Epoch [3/5], Step [200/600], Loss: 0.0872, Accuracy: 95.00%
Epoch [3/5], Step [300/600], Loss: 0.0115, Accuracy: 100.00%
Epoch [3/5], Step [400/600], Loss: 0.0526, Accuracy: 98.00%
Epoch [3/5], Step [500/600], Loss: 0.0

En each epoch, we iterate over train_loader using enumerate and passing images to it. We don't call _model.forward(images)_, the nn.Module already knows that forwars needs to be called when executing model(images). The output is then passed to the CrossEntropyLoss function. 

Next we perform backpropagation and optimization. In order to do this, we make the gradients zero (with the zero_grad() function), and then we call loss.backward() to perform the backpropagation. Finally, we call optimizer.step() to perform the Adam optimizer training step.

Then, we keep track of the accuracy of the training. The function torch.max() determines the prediction of the model (it returns the index of the maximum value in a tensor). The first argument is the tensor, and the second is the axis to evaluate. Axis = 0 is the batch_size dimension, and axis = 1 is the output node axis. The next line compares the predicted integers with the actual model. The function .item serves to access the value of the tensor (which is a tensor even after the application of sum()).

In [18]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 99.16 %


In order to test our model, we use the model.eval() function. The torch.no_grad() disables the autograd function, for sppeding up commputations. In thiscase, the iteration is over _test_loader_